In [ ]:
from ieasyhydro_sdk.sdk import IEasyHydroSDK, IEasyHydroHFSDK
from dotenv import load_dotenv

from ieasyhydro_sdk.filters import BasicDataValueFilters
import datetime as dt

import pandas as pd
import sys
import os

import holoviews as hv

hv.extension('bokeh')

# Get the absolute path of the directory containing the current script
cwd = os.getcwd()
print(cwd)
! pip freeze | grep ieasyhydro-sdk


# Construct the path to the iEasyHydroForecast directory
forecast_dir = os.path.join(
    cwd, '..', 'iEasyHydroForecast')

# Add the forecast directory to the Python path
sys.path.append(forecast_dir)

# Import the modules from the forecast library
import tag_library as tl
import forecast_library as fl

#env_file_path = "../config/.env_develop_kghm"
env_file_path = "../../../../sensitive_data_forecast_tools/config/.env_develop_kghm"
load_dotenv(env_file_path)
print("DEBUG: IEASYHYDRO_HOST: ", os.getenv("IEASYHYDRO_HOST"))
print("DEBUG: IEASYHYDROHF_HOST: ", os.getenv("IEASYHYDROHF_HOST"))
#print("DEBUG: IEASYHYDROHF_USERNAME: ", os.getenv("IEASYHYDROHF_USERNAME"))
#print("DEBUG: IEASYHYDROHF_PASSWORD: ", os.getenv("IEASYHYDROHF_PASSWORD"))

# Get data from hydrosolutions account on iEH HF
#iehhf_sdk_hsol = IEasyHydroHFSDK(
#    host='https://hf.ieasyhydro.org/api/v1/',
#    username=<username>,
#    password=<pwd>
#)

# Load sdk configuration from .env
ieh_sdk = IEasyHydroSDK(
    host='http://localhost:8881'
)
iehhf_sdk = IEasyHydroHFSDK()

today = dt.datetime.now()

predictor_dates = [dt.datetime(2024, 5, 13, 0, 0, 0), dt.datetime(2024, 5, 15, 12, 0, 0)]
predictor_dates = [dt.datetime(2024, 10, 18, 0, 0, 0), dt.datetime(2024, 10, 28, 12, 0, 0)]
predictor_dates = [today - dt.timedelta(days=3), today]

# Define date filter
filters = BasicDataValueFilters(
    local_date_time__gte=predictor_dates[0],
    local_date_time__lt=predictor_dates[1]
)



sites = ["15102", "15212", "15215", "15216", "15149", "15214", "15171",
                 "15194", "15189", "15292", "15261", "15954", "15960",
                 "16059", "16096", "16100", "16936"]
#sites = ["15030"]  # ["15194"]

for site in sites:
    #site = '15102'

    try:
        # Get data
        qdata = ieh_sdk.get_data_values_for_site(
            [site],
            'discharge_daily_average',
            filters=filters
        )
        qdata = pd.DataFrame(qdata['data_values'])
        #print("get_data_values_for_site:\n", qdata)
        #print(type(qdata))

        tdata = ieh_sdk.get_data_values_for_site(
            [site],
            'discharge_daily',
            filters=filters,
        )
        tdata = pd.DataFrame(tdata['data_values'])
        #print(tdata)

        # Get the first row from tdata in the wide format
        row = pd.DataFrame(tdata.iloc[-1]).transpose()
        #print(row)

        # add the row to qdata
        qdata = pd.concat([qdata, row])
        print(qdata.tail())
        print(f"{site} : {round(sum(qdata['data_value']), 3)}")

    except:
        print(f"site {site} not in DB")
        continue

In [ ]:
sites_kg_old_raw = ieh_sdk.get_discharge_sites()
#print(sites_kg_old_raw)
sites_kg_old = []
for site in sites_kg_old_raw:
    sites_kg_old.append(site['site_code'])
print(sites_kg_old)

In [ ]:
#sits_hsol = iehhf_sdk_hsol.get_discharge_sites()
#print(sits_hsol)

In [ ]:
# Test getting data from hsol account on iEH HF
#pentadal_norm = iehhf_sdk_hsol.get_norm_for_site("02268", "discharge", norm_period="p")
#print(f'pentad: {pentadal_norm}')
#decadal_norm = iehhf_sdk_hsol.get_norm_for_site("02268", "discharge")
#print(f'decad: {decadal_norm}')
#monthly_norm = iehhf_sdk_hsol.get_norm_for_site("02268", "discharge", norm_period="m")
#print(f'monthly: {monthly_norm}')

In [ ]:
test = iehhf_sdk.get_discharge_sites()
print(test[0])

In [ ]:
# Test get norm data
# Get decadal norm (default)
decadal_norm = iehhf_sdk.get_norm_for_site("16153", "discharge")
print(decadal_norm)
# Get monthly norm
monthly_norm = iehhf_sdk.get_norm_for_site("16153", "discharge", norm_period="m")
print(monthly_norm)
# Get pentadal norm
pentadal_norm = iehhf_sdk.get_norm_for_site("16153", "discharge", norm_period="p")
print(pentadal_norm)

In [ ]:
# Get site_code, site_name and region into a data frame
test_df = pd.DataFrame(test)[['site_code', 'official_name', 'basin', 'site_type']]
test_df['basin'] = test_df['basin'].apply(lambda x: x['official_name'])
test_df

In [ ]:
# Write the data frame to a csv file
#test_df.to_csv('discharge_sites.csv', index=False)

In [ ]:
# Use hvplot to plot data_value against local_date_time
plot = hv.Curve(data=qdata, kdims=['local_date_time'], vdims=['data_value'])
plot


In [ ]:
# Test SDK library to get static data from iEH HF
print("ieasyhydro_host: ", os.getenv("IEASYHYDRO_HOST"))
print("ieasyhydrohf_host: ", os.getenv("IEASYHYDROHF_HOST"))
ieasyhydro_hf_sdk = IEasyHydroHFSDK()

discharge_sites_data = ieasyhydro_hf_sdk.get_discharge_sites()
test_site = pd.DataFrame(discharge_sites_data).iloc[0]
print(test_site)

In [ ]:
test_site_basin_id = test_site['basin']
print(test_site_basin_id['official_name'])

In [ ]:
virtual_sites_data = ieasyhydro_hf_sdk.get_virtual_sites()
print(pd.DataFrame(virtual_sites_data).iloc[0])

In [ ]:
# Get meteo sites
meteo_sites = ieh_sdk.get_meteo_sites()
meteo_sites = pd.DataFrame(meteo_sites)
print(meteo_sites['site_code'])
print(type(meteo_sites['site_code'][0]))

In [ ]:
# Get station meta data
station_meta = ieh_sdk.get_data_values_for_site(
    ["15212"],
    'decade_precipitation',
    filters=filters
)
print(station_meta['variable'])
station_meta_df = pd.DataFrame([station_meta['variable']])
station_meta_df

In [ ]:
# Get weather station data
decade_precip = ieh_sdk.get_data_values_for_site(
    ["15212"],
    'decade_precipitation',
    filters=filters
)
decade_precip = pd.DataFrame(decade_precip['data_values'])
decade_precip

In [ ]:
monthly_precip = ieh_sdk.get_data_values_for_site(
    ["15212"],
    'monthly_precipitation',
    filters=filters
)
monthly_precip = pd.DataFrame(monthly_precip['data_values'])
monthly_precip

In [ ]:
# Get all sites from iEH DB
meteo_sites_data = ieh_sdk.get_meteo_sites()
meteo_sites_data = pd.DataFrame(meteo_sites_data)
# Filter for basin == "Чу"
meteo_sites_data = meteo_sites_data[meteo_sites_data['basin'] == "Чу"]
print(meteo_sites_data)

